<a href="https://colab.research.google.com/github/vilsonrodrigues/DataScienceEduc/blob/master/Gerador_de_notas_em_lista_questao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pré-Processamento

Importando as bibliotecas

In [0]:
import pandas as pd
import numpy as np

Importando os CSV's

In [0]:
situacoes_url = ["https://raw.githubusercontent.com/repitta/CienciaDeDadosEducacionais/master/arquivosCSV/lopSituacao2017.2.csv",
                 "https://raw.githubusercontent.com/repitta/CienciaDeDadosEducacionais/master/arquivosCSV/lopSituacao2018.1.csv",
                 "https://raw.githubusercontent.com/repitta/CienciaDeDadosEducacionais/master/arquivosCSV/lopSituacao2018.2.csv",
                 "https://raw.githubusercontent.com/repitta/CienciaDeDadosEducacionais/master/arquivosCSV/lopSituacao2019.1.csv",
                 "https://raw.githubusercontent.com/repitta/CienciaDeDadosEducacionais/master/arquivosCSV/lopSituacao2019.2.csv"]

usuarios_lop_url = "https://raw.githubusercontent.com/repitta/CienciaDeDadosEducacionais/master/arquivosCSV/lop_db_2019_09_csv_tratados/lopUsuarioTurma.csv"
 
submissoes_url   = "https://github.com/repitta/CienciaDeDadosEducacionais/raw/master/arquivosCSV/lop_db_2019_09_csv_tratados/lopSubLista.csv"                    

Lendo como CSV

In [0]:
#De 2017.2 até 2019.2
situacoes2017_2 = pd.read_csv(situacoes_url[0], sep=',')
situacoes2018_1 = pd.read_csv(situacoes_url[1], sep=',')
situacoes2018_2 = pd.read_csv(situacoes_url[2], sep=',')
situacoes2019_1 = pd.read_csv(situacoes_url[3], sep=',')
situacoes2019_2 = pd.read_csv(situacoes_url[4], sep=',')

# Submissoes dos alunos no LOP

submissoes = pd.read_csv(submissoes_url, sep=',')

#Usuarios cadastrados no LOP 
usuarios_lop = pd.read_csv(usuarios_lop_url, sep=',')

Inserindo numa lista

In [0]:
situacoes = [situacoes2017_2,situacoes2018_1, situacoes2018_2, situacoes2019_1, situacoes2019_2]

In [0]:
tam = len(situacoes)

Tratando os campos de situação

In [0]:
for i in range(tam):
  situacoes[i] = situacoes[i].drop(columns="Unnamed: 0")
  situacoes[i].fillna(0, inplace=True)
  situacoes[i]["matricula"] = situacoes[i]["matricula"].astype(str)
  situacoes[i]= situacoes[i].rename(columns={"ano": "anoSigaa"})
  situacoes[i]["anoSigaa"] = situacoes[i]["anoSigaa"].astype(float)
  situacoes[i]["turma"] = situacoes[i]["turma"].astype(str)

Tratando o campo das submissões

In [0]:
submissoes = submissoes.drop(columns=["Unnamed: 0","index"])  
submissoes.fillna(0, inplace=True)
submissoes.drop(submissoes[submissoes["data"]=="0"].index, inplace=True)
submissoes.drop(submissoes[submissoes["titulo"]=="0"].index, inplace=True)
submissoes["data"] = pd.to_datetime(submissoes["data"], format = "%Y/%m/%dT%H:%M:%S")
submissoes = submissoes.rename(columns={"quantidade": "quantidadeTotalLista"})

Trantando o campo dos usuários

In [0]:
usuarios_lop = usuarios_lop.drop(columns="Unnamed: 0")
usuarios_lop.fillna(0, inplace=True)
usuarios_lop = usuarios_lop.rename(columns={"descricaoTurma": "turma"})
usuarios_lop["turma"] =  usuarios_lop["turma"].str.split(expand=True)[5]
usuarios_lop["matricula"] = usuarios_lop["matricula"].astype(str)
usuarios_lop= usuarios_lop.rename(columns={"ano": "anoLop"})
usuarios_lop["anoLop"] = usuarios_lop["anoLop"].astype(float)
usuarios_lop["turma"] = usuarios_lop["turma"].astype(str)


Colunas de situação, usuário e submissão

In [9]:
print(situacoes[0].columns)
print(" ")
print(usuarios_lop.columns)
print(" ")
print(submissoes.columns)

Index(['anoSigaa', 'turma', 'matricula', 'nota', 'situacao'], dtype='object')
 
Index(['idTurma', 'anoLop', 'turma', 'matricula', 'idAluno', 'nome'], dtype='object')
 
Index(['idLista', 'questao', 'titulo', 'quantidadeTotalLista', 'data',
       'porcentagemAcerto', 'idAluno'],
      dtype='object')


Concatenando as tabelas de situações em um só

In [0]:
a =  pd.concat([situacoes[0], situacoes[1], situacoes[2], situacoes[3],situacoes[4]], ignore_index=True, axis=0)

Juntando situação de usuários

In [0]:
mergeSituacoeUsuarios = pd.merge(right= a, left = usuarios_lop, on=["matricula","turma"], how="outer")

Limpar a tabela para ficar só com as ocorrencias de usuarios que estão cadastrados no Lop

In [12]:
mergeSituacoeUsuarios = mergeSituacoeUsuarios.loc[mergeSituacoeUsuarios["anoSigaa"] == mergeSituacoeUsuarios["anoLop"] ]
mergeSituacoeUsuarios.head(1)

,idTurma,anoLop,turma,matricula,idAluno,nome,anoSigaa,nota,situacao
0,598e15296d8650eb27d52e3d,2017.2,01A,20170106538,599b051d495fac076a06d3f5,Aleff Ramon da Silva Costa,2017.2,7.1,APROVADO


Função para criar um campo extra na tabela submissoes para ter certeza que a submissão pertence ao aluno matriculado naquele semestre

In [0]:
def anosub(data):
  if data.month >= 8:
    anoSubmissao = data.year + 0.2
  else :
    anoSubmissao = data.year + 0.1
  return anoSubmissao

In [0]:
submissoes["anoSubmissao"] = submissoes["data"].apply(anosub)

Juntando a tabela situacaoUsuario com a tabela de submissões

In [0]:
mergeSituacoeUsuariosSubLista = pd.merge(right=mergeSituacoeUsuarios, left = submissoes, on=["idAluno"], how="outer")

Ficando apenas com tem o ano do sigaa igual ao da submissão

In [16]:
mergeSituacoeUsuariosSubLista = mergeSituacoeUsuariosSubLista.loc[mergeSituacoeUsuariosSubLista["anoSigaa"] == mergeSituacoeUsuariosSubLista["anoSubmissao"] ]
mergeSituacoeUsuariosSubLista.head(1)

,idLista,questao,titulo,quantidadeTotalLista,data,porcentagemAcerto,idAluno,anoSubmissao,idTurma,anoLop,turma,matricula,nome,anoSigaa,nota,situacao
148,598cd57a3bc48946977dcf67,59885f8d684f4a3f78163c00,Lista de Laboratório 1 - Expressões Aritmética...,5.0,2017-08-14 16:29:10.522000+00:00,100.0,5991cb910be9d42f2f68b906,2017.2,598e15296d8650eb27d52e3d,2017.2,01A,20170039453,Amanda,2017.2,7.9,APROVADO


#Função gera .xls

In [0]:
def geraXls(table,nomeDaLista,turmas):
  table["data"] = table["data"].astype(str)
  table.to_excel(r"Nota"+nomeDaLista+"Turmas"+turmas+'.xls')

# Função para buscar desempenho dos alunos em uma questão de uma lista

In [18]:
tabelaSub = mergeSituacoeUsuariosSubLista.iloc[:,[1,4,5,6,10,11,12,13]]
tabelaSub.head()

,questao,data,porcentagemAcerto,idAluno,turma,matricula,nome,anoSigaa
148,59885f8d684f4a3f78163c00,2017-08-14 16:29:10.522000+00:00,100.0,5991cb910be9d42f2f68b906,01A,20170039453,Amanda,2017.2
149,59887121d7c3703f96fda00c,2017-08-14 16:50:36.546000+00:00,100.0,5991cb910be9d42f2f68b906,01A,20170039453,Amanda,2017.2
150,59887c83684f4a3f78163c02,2017-08-14 17:32:49.320000+00:00,100.0,5991cb910be9d42f2f68b906,01A,20170039453,Amanda,2017.2
151,59888cb2d2f43b3f548ef054,2017-08-15 00:55:35.417000+00:00,100.0,5991cb910be9d42f2f68b906,01A,20170039453,Amanda,2017.2
152,59888ea5d2f43b3f548ef057,2017-08-15 00:20:39.814000+00:00,100.0,5991cb910be9d42f2f68b906,01A,20170039453,Amanda,2017.2


In [0]:
def desempenhoNaQuestao(table,questao,turma,ano,nomeDaLista):
  tab = pd.DataFrame()
  tab = table.loc[(table["questao"] == questao) & (table["anoSigaa"] == ano) & (table["turma"].str.contains(turmas)),:]
  tab = tab.sort_values(by=["turma","nome"])
  geraXls(tab,nomeDaLista,turmas)
  return tab

In [0]:
turmas = "03"
nomeDaLista = "Lista03"
questao = "59a17f695e65ea64a07f26ea"
ano = 2019.2
porcentagemAcerto = 100.0

Chamada da função

In [0]:
d = desempenhoNaQuestao(tabelaSub,questao,turmas,ano,nomeDaLista)

In [0]:
d.loc[d["matricula"]=="20190081445",:]

,questao,data,porcentagemAcerto,idAluno,turma,matricula,nome,anoSigaa
49479,59a17f695e65ea64a07f26ea,2019-09-11 13:23:09.938000+00:00,100.0,5d4b12cec1853254c20afbb7,03B,20190081445,Syllas do Nascimento,2019.2
49480,59a17f695e65ea64a07f26ea,2019-09-11 13:23:10.727000+00:00,100.0,5d4b12cec1853254c20afbb7,03B,20190081445,Syllas do Nascimento,2019.2
49481,59a17f695e65ea64a07f26ea,2019-09-11 13:23:12.708000+00:00,100.0,5d4b12cec1853254c20afbb7,03B,20190081445,Syllas do Nascimento,2019.2
49482,59a17f695e65ea64a07f26ea,2019-09-11 13:30:13.391000+00:00,100.0,5d4b12cec1853254c20afbb7,03B,20190081445,Syllas do Nascimento,2019.2


# Montagem das notas dos alunos por lista

In [0]:
a = mergeSituacoeUsuariosSubLista[["titulo","quantidadeTotalLista"]]
a.drop_duplicates(subset="titulo", keep='first', inplace=True)
a.sort_values(by="titulo")

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,titulo,quantidadeTotalLista
5044,Lista 1.0 - Expressões aritméticas,10.0
5067,Lista 1.2 - Funções aritméticas,9.0
181,Lista de Exercícios 1 - Expressões e variáveis,9.0
140,Lista de Exercícios 2 - Estruturas Condicionais,12.0
154,Lista de Exercícios 3 - Laço Condicional,12.0
188,Lista de Exercícios 4 - Laço contado,9.0
220,Lista de Exercícios 4.1 - Problemas tradiciona...,8.0
208,Lista de Exercícios 5 - Vetores,11.0
132,Lista de Laboratório 1 - Expressões Aritmética...,4.0
137,Lista de Laboratório 2 - Expressões Lógicas e ...,7.0


**Porcentagem Máxima por aluno em cada questão da lista**

In [0]:
NotaMaxQuestao = mergeSituacoeUsuariosSubLista.groupby(["idLista","idAluno","questao","anoSubmissao"]).max()
NotaMaxQuestao.rename(columns={"porcentagemAcerto":"porcentagemMaxima"}, inplace=True)
NotaMaxQuestao = NotaMaxQuestao.reset_index()
print(NotaMaxQuestao.columns)
NotaMaxQuestao.head(1)

Index(['idLista', 'idAluno', 'questao', 'anoSubmissao', 'titulo',
       'quantidadeTotalLista', 'data', 'porcentagemMaxima', 'idTurma',
       'anoLop', 'turma', 'matricula', 'nome', 'situacao', 'anoSigaa'],
      dtype='object')


,idLista,idAluno,questao,anoSubmissao,titulo,quantidadeTotalLista,data,porcentagemMaxima,idTurma,anoLop,turma,matricula,nome,situacao,anoSigaa
0,598cd57a3bc48946977dcf67,5991cb910be9d42f2f68b906,59885f8d684f4a3f78163c00,2017.2,Lista de Laboratório 1 - Expressões Aritmética...,4.0,2017-08-14 16:29:10.522000+00:00,100.0,598e15296d8650eb27d52e3d,2017.2,01A,20170039453,Amanda,APROVADO,2017.2


**Quantidade de questoes feitas por aluno em cada lista**

In [0]:
quantidade = NotaMaxQuestao.groupby(["titulo","idAluno","anoSubmissao"]).count()
quantidade.rename(columns={"quantidadeTotalLista":"questoesFeitas"}, inplace=True)
quantidade= quantidade.reset_index()
quantidade= quantidade[["titulo","idAluno","anoSubmissao","questoesFeitas"]]
print(quantidade.columns)
quantidade.head(1)

Index(['titulo', 'idAluno', 'anoSubmissao', 'questoesFeitas'], dtype='object')


,titulo,idAluno,anoSubmissao,questoesFeitas
0,Lista 1.0 - Expressões aritméticas,59b91272570dda40b535c50d,2019.2,10


**Nota final dos alunos por lista**

In [0]:
NotaLista = NotaMaxQuestao.groupby(["titulo","idAluno","anoSubmissao"]).sum()
NotaLista.rename(columns={"porcentagemMaxima":"SomaLista"}, inplace=True)
#NotaLista = NotaLista.drop('ano',axis=1)
NotaLista = NotaLista.reset_index()
NotaLista = NotaLista[["titulo",'idAluno',"anoSubmissao",'SomaLista']]
print(NotaLista.columns)
NotaLista.head(1)

Index(['titulo', 'idAluno', 'anoSubmissao', 'SomaLista'], dtype='object')


,titulo,idAluno,anoSubmissao,SomaLista
0,Lista 1.0 - Expressões aritméticas,59b91272570dda40b535c50d,2019.2,1000.0


In [0]:
mergeQuanNotaLista =  pd.merge(right= quantidade, left = NotaLista, on=['idAluno','titulo',"anoSubmissao"], how="inner")
print(mergeQuanNotaLista.columns)
print(mergeQuanNotaLista.shape)
mergeQuanNotaLista.head(1)

Index(['titulo', 'idAluno', 'anoSubmissao', 'SomaLista', 'questoesFeitas'], dtype='object')
(7926, 5)


,titulo,idAluno,anoSubmissao,SomaLista,questoesFeitas
0,Lista 1.0 - Expressões aritméticas,59b91272570dda40b535c50d,2019.2,1000.0,10


**Quantidade de questoes por lista**

In [0]:
quantQuesList = mergeSituacoeUsuariosSubLista[['quantidadeTotalLista','titulo']]
quantQuesList =quantQuesList.drop_duplicates()
print(quantQuesList.shape)
quantQuesList

(24, 2)


,quantidadeTotalLista,titulo
132,4.0,Lista de Laboratório 1 - Expressões Aritmética...
136,9.0,Lista de Revisão Unidade 1
137,7.0,Lista de Laboratório 2 - Expressões Lógicas e ...
140,12.0,Lista de Exercícios 2 - Estruturas Condicionais
154,12.0,Lista de Exercícios 3 - Laço Condicional
156,7.0,Lista de Laboratório 3 - Estruturas de Repeti...
167,5.0,Prova 1 - Turma 09 - 2017.1
179,5.0,Lista de Laboratório 4 - Repetição Contada
181,9.0,Lista de Exercícios 1 - Expressões e variáveis
188,9.0,Lista de Exercícios 4 - Laço contado


**Concatenando as informações das tabelas mergeQuanNotaLista e quantidade de questoes por lista para calcular o nota final de cada lista**

In [0]:
mergeQuanNotaListaQuanQ = mergeQuanNotaLista.join(quantQuesList.set_index('titulo'), on='titulo')
mergeQuanNotaListaQuanQ["NotaLista"]= mergeQuanNotaListaQuanQ["SomaLista"] /mergeQuanNotaListaQuanQ['quantidadeTotalLista']
print(mergeQuanNotaListaQuanQ.columns)
print(mergeQuanNotaListaQuanQ.shape)
mergeQuanNotaListaQuanQ.head(3)

Index(['titulo', 'idAluno', 'anoSubmissao', 'SomaLista', 'questoesFeitas',
       'quantidadeTotalLista', 'NotaLista'],
      dtype='object')
(7926, 7)


,titulo,idAluno,anoSubmissao,SomaLista,questoesFeitas,quantidadeTotalLista,NotaLista
0,Lista 1.0 - Expressões aritméticas,59b91272570dda40b535c50d,2019.2,1000.0,10,10.0,100.0
1,Lista 1.0 - Expressões aritméticas,5b7b1f4f9513a60408ac2d21,2019.2,1000.0,10,10.0,100.0
2,Lista 1.0 - Expressões aritméticas,5c77f58d7776520411e9aff4,2019.1,1000.0,10,10.0,100.0


In [0]:
mergeSituacoeUsuariosSubLista.columns

Index(['idLista', 'questao', 'titulo', 'quantidadeTotalLista', 'data',
       'porcentagemAcerto', 'idAluno', 'anoSubmissao', 'idTurma', 'anoLop',
       'turma', 'matricula', 'nome', 'situacao', 'anoSigaa'],
      dtype='object')

**Tabela final com todas as informações do aluno das listas**

In [0]:
mergeSituacoeUsuariosSubListaM = mergeSituacoeUsuariosSubLista[['idAluno','titulo', 'anoSubmissao','anoLop', 'turma', 'matricula', 'nome', 'situacao', 'anoSigaa']]
tabelaFinalListas= mergeQuanNotaListaQuanQ.join(mergeSituacoeUsuariosSubListaM.set_index(['idAluno','titulo', 'anoSubmissao']), on=['idAluno','titulo', 'anoSubmissao'])
#remover as linhas duplicadas
tabelaFinalListas = tabelaFinalListas.drop_duplicates()
tabelaFinalListas.head()


,titulo,idAluno,anoSubmissao,SomaLista,questoesFeitas,quantidadeTotalLista,NotaLista,anoLop,turma,matricula,nome,situacao,anoSigaa
0,Lista 1.0 - Expressões aritméticas,59b91272570dda40b535c50d,2019.2,1000.0,10,10.0,100.0,2019.2,03A,2016012150,João,MATRICULADO,2019.2
1,Lista 1.0 - Expressões aritméticas,5b7b1f4f9513a60408ac2d21,2019.2,1000.0,10,10.0,100.0,2019.2,03D,20180034595,Eduardo Freire de Almeida,MATRICULADO,2019.2
2,Lista 1.0 - Expressões aritméticas,5c77f58d7776520411e9aff4,2019.1,1000.0,10,10.0,100.0,2019.1,02D,20180146250,Vitor da Rocha Silva,REPROVADO,2019.1
3,Lista 1.0 - Expressões aritméticas,5c77fa897776520411e9b01c,2019.2,1000.0,10,10.0,100.0,2019.2,02C,20180044170,Wendson Canuto Bezerra Dos Santos,MATRICULADO,2019.2
4,Lista 1.0 - Expressões aritméticas,5c87bd6923798d040d0270ad,2019.2,1000.0,10,10.0,100.0,2019.2,03D,20180135195,José Humberto Valente da Silva Junior,MATRICULADO,2019.2


**Exportar a tabela final das notas das listas para um arquivo CSV**

In [0]:
 tabelaFinalListas.to_csv(r'notasListas.csv')

In [0]:
def tabelaNotaLista(ano, titulo, turma): 
  tabela = tabelaFinalListas.loc[ (tabelaFinalListas["anoSigaa"] == ano) & (tabelaFinalListas["titulo"].str.contains(titulo)) & (tabelaFinalListas["turma"].str.contains(turma)) ,: ]
  tabela = tabela[["anoSigaa","matricula","turma", "nome", "NotaLista","SomaLista"]].sort_values(by="nome")
  return tabela

**Exportando o dataframe para um arquivo excel**

In [0]:
turmas= ["03A","03B","03C","03D"]
listas = ["Lista 1.2","Estruturas Condicionais"]
for t in turmas:
  for l in listas:
    tab = tabelaNotaLista(2019.2, l, t)
    tab =tab[["anoSigaa","matricula","turma", "nome", "NotaLista"]].sort_values(by="nome")
    tab.to_excel(r'Nota'+l+" "+t+'.xls')

In [0]:
#Fiz essa lista separa pq o professor não cobrou todas as questoes para valer nota .. apenas 10 questoes das 15 que tem na lista
turmas= ["03A","03B","03C","03D"]
for t in turmas:
    tab = tabelaNotaLista(2019.2, "3 - La", t)
    tab["NotaLista"] = tab["SomaLista"] / 10
    tab =tab[["anoSigaa","matricula","turma", "nome", "NotaLista"]].sort_values(by="nome")
    tab.to_excel(r'NotaLista03'+" "+t+'.xls')

Tabela com notas por lista

In [0]:
tabelaFinalListas.loc[tabelaFinalListas["idAluno"] =="59b91272570dda40b535c50d",:]

,titulo,idAluno,anoSubmissao,SomaLista,questoesFeitas,quantidadeTotalLista,NotaLista,anoLop,turma,matricula,nome,situacao,anoSigaa
0,Lista 1.0 - Expressões aritméticas,59b91272570dda40b535c50d,2019.2,1000.0,10,10.0,100.000000,2019.2,03A,2016012150,João,MATRICULADO,2019.2
89,Lista 1.2 - Funções aritméticas,59b91272570dda40b535c50d,2019.2,900.0,9,9.0,100.000000,2019.2,03A,2016012150,João,MATRICULADO,2019.2
821,Lista de Exercícios 2 - Estruturas Condicionais,59b91272570dda40b535c50d,2019.2,700.0,7,12.0,58.333333,2019.2,03A,2016012150,João,MATRICULADO,2019.2
1616,Lista de Exercícios 3 - Laço Condicional,59b91272570dda40b535c50d,2019.2,300.0,3,12.0,25.000000,2019.2,03A,2016012150,João,MATRICULADO,2019.2
4033,Lista de Laboratório 2 - Expressões Lógicas e ...,59b91272570dda40b535c50d,2019.2,700.0,7,7.0,100.000000,2019.2,03A,2016012150,João,MATRICULADO,2019.2
4884,Lista de Laboratório 3 - Estruturas de Repeti...,59b91272570dda40b535c50d,2019.2,500.0,5,7.0,71.428571,2019.2,03A,2016012150,João,MATRICULADO,2019.2
6676,Lista de Revisão Unidade 1,59b91272570dda40b535c50d,2019.2,200.0,2,9.0,22.222222,2019.2,03A,2016012150,João,MATRICULADO,2019.2
